In [1]:
from unstructured.partition.pdf import partition_pdf

partitions = partition_pdf(
    filename='../data/Lunch2024.pdf',
    max_characters=10000,
    new_after_n_chars=6000
)

/Users/michaeleco/Documents/Github/project-avery/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
partitions

In [3]:
for element in partitions:
    print(f'{element.category}: {element}')
    print('-----')

UncategorizedText: Est. 2000
-----
Title: lunch menu
-----
NarrativeText: Fratellino in Italian means “little brother.” Welcome to the little brother’s restaurant, an homage to the older brothers’ teachings and support that inspired owner Cesar Lomeli to first venture out and create an Italian culinary experience in Orange County. As a restaurant, we were the little brother of the community when we started in 2000, and with your support and patronage, we grew up to be the restaurant scrupulously committed to quality that you visit today. Fratellino’s is our thanks to you and a continued promise to provide you mouthwatering food, diligent service, and not only a seat in our restaurant, but also a place in our family. The Lomeli Family
-----
Title: Gift Cards
-----
NarrativeText: A Fratellino’s Gift Card is the perfect way to say “thank you,” “happy birthday,” “congratulations,” and any other well wishes to friends, family, and coworkers. Gift cards are available in any denomination. Jus

In [4]:
texts = []
for element in partitions:
    texts.append(element.text)
print(texts)

['Est. 2000', 'lunch menu', 'Fratellino in Italian means “little brother.” Welcome to the little brother’s restaurant, an homage to the older brothers’ teachings and support that inspired owner Cesar Lomeli to first venture out and create an Italian culinary experience in Orange County. As a restaurant, we were the little brother of the community when we started in 2000, and with your support and patronage, we grew up to be the restaurant scrupulously committed to quality that you visit today. Fratellino’s is our thanks to you and a continued promise to provide you mouthwatering food, diligent service, and not only a seat in our restaurant, but also a place in our family. The Lomeli Family', 'Gift Cards', 'A Fratellino’s Gift Card is the perfect way to say “thank you,” “happy birthday,” “congratulations,” and any other well wishes to friends, family, and coworkers. Gift cards are available in any denomination. Just let your server know that you would like to take one home today!', 'WWW

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever

vectorstore = Chroma(
    collection_name='test',
    embedding_function=OllamaEmbeddings(model='nomic-embed-text'),
    persist_directory='../data/chroma'
)

store = InMemoryStore()
id_key = 'doc_id'

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key
)

/var/folders/17/504r7zwd36jcb_pwxly46r940000gn/T/ipykernel_56902/2207982430.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [6]:
from uuid import uuid4
from langchain.schema.document import Document

doc_ids = [str(uuid4()) for _ in texts]
document_texts = [
    Document(page_content=text, metadata={id_key: doc_ids[i]}) for i, text in enumerate(texts)
]

retriever.vectorstore.add_documents(document_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))